In [37]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib as plt
from collections import defaultdict
import math
import matplotlib.pyplot as plt

In [38]:
def get_data(portfolio, start_date, end_date):
    # Initialize an empty DataFrame
    data = pd.DataFrame()

    # Loop through each country in the portfolio
    for tickers in portfolio:
        # Download data for each ticker
        df = yf.download(tickers, start=start_date, end=end_date)

        # Concatenate the data to the main DataFrame
        data = pd.concat([data, df], axis=1)

    return data

In [39]:
START_DATE = "1984-01-01"
END_DATE = "2024-04-26"
columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'] # total data
features = ['Open', 'Close', 'High', 'Low', 'Volume'] # input features
portfolio = ['^GSPC']

In [40]:
data = get_data(portfolio, start_date=START_DATE, end_date=END_DATE)
data.interpolate(method = 'linear', inplace = True,limit_direction = 'both')

[*********************100%%**********************]  1 of 1 completed


In [41]:
jan_to_april = data[(data.index.month >= 1) & (data.index.month <= 4)]
july_to_oct = data[(data.index.month >= 8) & (data.index.month <= 10) | ((data.index.month == 7) & (data.index.day == 31))]


In [42]:
def filter_by_yr(df, start_yr, end_yr):
	dfs_by_year = {}

	for year in range(start_yr, end_yr + 1):
		dfs_by_year[year] = df[df.index.year == year]

	return dfs_by_year

In [43]:
jan_to_april_by_yr = filter_by_yr(jan_to_april, 1984, 2024)
july_to_oct_by_yr = filter_by_yr(july_to_oct, 1984, 2024)

In [44]:
def generate_labels(df, start_yr, end_yr):
	labels = []

	for year in range(start_yr, end_yr + 1):
		curr = df[year]
		diff = curr["Adj Close"].iloc[-1] - curr["Adj Close"].iloc[0]
		print("Year " + str(year) + ": " + str(diff))
		labels.append(1 if diff > 0 else 0)

	return labels

In [45]:
labels = generate_labels(july_to_oct_by_yr, 1984, 2023)
labels.append("NA")

Year 1984: 15.42999267578125
Year 1985: -1.0999908447265625
Year 1986: 7.8600006103515625
Year 1987: -66.87001037597656
Year 1988: 6.760009765625
Year 1989: -5.720001220703125
Year 1990: -52.149993896484375
Year 1991: 4.6400146484375
Year 1992: -5.529998779296875
Year 1993: 17.67999267578125
Year 1994: 11.339996337890625
Year 1995: 19.44000244140625
Year 1996: 65.32000732421875
Year 1997: -39.69000244140625
Year 1998: -22.0
Year 1999: 34.8800048828125
Year 2000: -1.429931640625
Year 2001: -151.449951171875
Year 2002: -25.8599853515625
Year 2003: 60.39996337890625
Year 2004: 23.5799560546875
Year 2005: -28.3399658203125
Year 2006: 101.2799072265625
Year 2007: 94.1099853515625
Year 2008: -298.6300048828125
Year 2009: 48.7099609375
Year 2010: 57.4000244140625
Year 2011: -33.639892578125
Year 2012: 32.840087890625
Year 2013: 70.81005859375
Year 2014: 87.3800048828125
Year 2015: -24.47998046875
Year 2016: -44.690185546875
Year 2017: 104.9599609375
Year 2018: -104.550048828125
Year 2019: 57.

In [46]:
jan_to_april_by_yr[2009]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,902.989990,934.729980,899.349976,931.799988,931.799988,4048270000
2009-01-05,929.169983,936.630005,919.530029,927.450012,927.450012,5413910000
2009-01-06,931.169983,943.849976,927.280029,934.700012,934.700012,5392620000
2009-01-07,927.450012,927.450012,902.369995,906.650024,906.650024,4704940000
2009-01-08,905.729980,910.000000,896.809998,909.729980,909.729980,4991550000
...,...,...,...,...,...,...
2009-04-24,853.909973,871.799988,853.909973,866.229980,866.229980,7114440000
2009-04-27,862.820007,868.830017,854.650024,857.510010,857.510010,5613460000
2009-04-28,854.479980,864.479980,847.119995,855.159973,855.159973,6328000000


In [47]:
idx = [i for i in range(1984, 2025)]
#np.empty((len(idx), 85)) * np.nan
df = pd.DataFrame()

In [48]:
for year in idx:
	row = np.array(jan_to_april_by_yr[year]["Adj Close"])
	df[year] = pd.Series(row)

In [49]:
df = df.T

def fill_with_row_median(row):
    median = row.median()
    return row.fillna(median)

# Apply the function across the rows (axis=1)
df = df.apply(fill_with_row_median, axis=1)

In [50]:
col_list = ["S&P_datapoint_{}".format(i) for i in range(1, 84)]

In [51]:
df.columns = col_list

In [52]:
df["label"] = labels

In [53]:
df.to_csv("clean/s&p_data.csv", index = True)

In [54]:
df

,S&P_datapoint_1,S&P_datapoint_2,S&P_datapoint_3,S&P_datapoint_4,S&P_datapoint_5,S&P_datapoint_6,S&P_datapoint_7,S&P_datapoint_8,S&P_datapoint_9,S&P_datapoint_10,...,S&P_datapoint_75,S&P_datapoint_76,S&P_datapoint_77,S&P_datapoint_78,S&P_datapoint_79,S&P_datapoint_80,S&P_datapoint_81,S&P_datapoint_82,S&P_datapoint_83,label
1984,164.039993,166.779999,168.809998,169.279999,168.899994,167.949997,167.800003,167.750000,167.020004,167.179993,...,158.970001,157.899994,158.020004,156.800003,158.070007,158.649994,160.300003,159.889999,160.050003,1
1985,165.369995,164.570007,163.679993,164.240005,163.990005,165.179993,168.309998,167.910004,170.509995,170.809998,...,180.839996,181.110001,180.699997,181.880005,182.259995,183.429993,182.179993,180.630005,179.830002,0
1986,209.589996,210.880005,210.649994,213.800003,207.970001,206.110001,205.960007,206.720001,206.639999,208.259995,...,242.380005,244.740005,242.419998,241.750000,242.020004,242.289993,243.080002,240.509995,235.520004,1
1987,246.449997,252.190002,252.779999,255.330002,257.279999,258.730011,260.299988,259.950012,262.640015,265.489990,...,286.089996,293.070007,287.190002,286.820007,281.519989,281.829987,282.510010,284.570007,288.359985,0
1988,255.940002,258.630005,258.890015,261.070007,243.399994,247.490005,245.419998,245.809998,245.880005,252.050003,...,257.920013,256.130005,256.420013,260.140015,262.510010,263.929993,263.799988,262.609985,261.329987,1
1989,275.309998,279.429993,280.010010,280.670013,280.980011,280.380005,282.010010,283.170013,283.869995,284.140015,...,307.149994,306.190002,309.609985,308.690002,306.750000,306.929993,309.579987,309.640015,293.899994,0
1990,359.690002,358.760010,355.670013,352.200012,353.790009,349.619995,347.309998,348.529999,339.929993,337.000000,...,340.720001,338.089996,335.119995,331.049988,330.359985,332.029999,332.920013,329.109985,330.799988,0
1991,326.450012,321.910004,321.000000,315.440002,314.899994,311.489990,314.529999,315.230011,312.489990,313.730011,...,388.459991,384.200012,380.950012,381.760010,382.760010,379.250000,379.019989,373.660004,375.339996,1
1992,417.260010,419.339996,417.959991,417.399994,418.100006,417.609985,415.100006,414.339996,420.440002,420.769989,...,416.040009,410.179993,410.260010,409.809998,411.600006,409.019989,408.450012,409.109985,412.019989,0
1993,435.380005,434.339996,434.519989,430.730011,429.049988,430.950012,431.040009,433.029999,435.940002,437.149994,...,445.100006,443.630005,439.459991,437.029999,433.540009,438.010010,438.019989,438.890015,440.190002,1
